<img src="image.jpg">

 # PyPRT - Building Modeling Optimization

This notebook is used to show how the Python bindings for PRT can be employed in optimizing the rule attributes of a generated model to achive predefined performance goals.

TODO:
    - change "total report for optimisation" to "building greenery (trees and ground greenery)" or something like that in Ioanna's rule
    - find a way to put a limit to the minimum thickness of the building walls
    - put different weight (<10) for the number of trees: total report for optimization = green area + 10* nber of trees in order to have a better results for the step 1 optimization
    
    - research for other python library that has optimiaztion algorithms with possibility to define input attributes step size

#### Imports and paths-related functions

In [1]:
import sys, os

sys.path.append(os.path.join(os.getcwd(), "..", "src"))
from utility import visualize_PRT_results, combine_reports

SDK_PATH = os.path.join(os.getcwd(), "..", "install", "bin")
sys.path.append(SDK_PATH)

import pyprt

from scipy import optimize
import numpy as np

CS_FOLDER = os.getcwd()
def asset_file(filename):
    return os.path.join(CS_FOLDER, filename)

from IPython.display import IFrame
from arcgis import GIS

In [27]:
my_gis = GIS(username="CLechot_zurich")

Enter password: ········


In [2]:
print("\nInitializing PRT.")
pyprt.initialize_prt(SDK_PATH)

if not pyprt.is_prt_initialized():
    raise Exception("PRT is not initialized")


Initializing PRT.
[PRT] [2019-09-24 13:43:51] [info] Esri Procedural Runtime
[PRT] [2019-09-24 13:43:51] [info] Version: 2.1.5704 | Build Date: Tue 09/03/2019 06:40 PM | Build Config: PRT_BC_REL PRT_CC_OPT PRT_TC_VC141 | OS: win32 | ARCH: x86_64
[PRT] [2019-09-24 13:43:51] [debug] prtPlugins item 0: 'C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\..\install\lib'
[PRT] [2019-09-24 13:43:51] [error] failed to load library: error while loading library 'C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\..\install\lib\com.esri.prt.unreal.dll': The specified module could not be found.

[PRT] [2019-09-24 13:43:51] [warning] failed to load library com.esri.prt.unreal.dll
[PRT] [2019-09-24 13:43:51] [error] failed to load library: error while loading library 'C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\..\install\lib\DatasmithSDK.dll': The specified module could not 

## Optimization Problem Definition

In this notebook, we are using a CGA rule that allows us to generate modern and sustainable buildings. The architecture of these buildings is focused on the green aspect of the building. Therefore, in addition to be modern and esthetical, the generated building answers also to the actual requirements of the city which are to encourage the integration of greenery on the buildings.

<img src="green_building_parameters.png" width="1000" height="100">

In [3]:
rpk_green = asset_file("envelope2409.rpk") #envelope1806.rpk
attrs_green = {'ruleFile' : "rules/typology/envelope2409.cgb", 'startRule' : "Default$Lot", 'report_but_not_display_green' : True}

shape_geo = asset_file("parcelN.obj")

### Optimization with envelope.rpk

Step 1 : optimize the green area based on the lot coverage parameter

### The variables

In [4]:
goal_str_green = "Greenery Area" + "_sum"

input_attr_green = ["lot_coverage_parameter"]
x0_green = [float(50)]

In [6]:
m = pyprt.ModelGenerator(shape_geo)
model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.examples.PyEncoder", {'emitGeometry' : False})

[PRT] [2019-09-24 13:46:39] [debug] trying to read initial shape geometry from file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5CparcelN.obj

[PRT] [2019-09-24 13:46:39] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope2409.rpk

[PRT] [2019-09-24 13:46:39] [debug] resolve map = <ResolveMap>
	<attribute key="assets/wall_plaster.jpg" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/wall_plaster.jpg" type="str"/>
	<attribute key="/green_optimization_3/assets/wall_plaster.jpg" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/wall_plaster.jpg" type="str"/>
	<attribute key="assets/lawn2.jpg" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk

In [7]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    generated_mod_green = m.generate_model(attrs_green)
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green[0])
        return -rep_green[goal_str_green]
    else:
        print("Error in optimization process.")

In [8]:
bounds_green = [(0,100)]

In [9]:
-fct_green(x0_green)

2168.6307416254404

In [10]:
import time
start = time.time()

res_green = optimize.differential_evolution(fct_green, bounds_green, seed = 666) # dual_annealing

duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

The optimization lasted: 27.992393016815186


     fun: -2196.7870170993974
 message: 'Optimization terminated successfully.'
    nfev: 32
     nit: 1
 success: True
       x: array([82.26263821])

### Results Visualization

In [14]:
export_file_name = "SLPKModel1"
enc_optionsSLPK = {'sceneType' : "Local",'baseName' : export_file_name,'sceneWkid' : "3857",'layerTextureEncoding' : ["2"],'layerEnabled' : [True],'layerUID' : ["1"],'layerName' : ["1"],'layerTextureQuality' : [1.0],'layerTextureCompression' : [9],'layerTextureScaling' : [1.0],'layerTextureMaxDimension' : [2048],'layerFeatureGranularity' : ["0"],'layerBackfaceCulling' : [False]}

attrs_green[input_attr_green[0]] = res_green.x[0]
green_model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.codecs.I3SEncoder", enc_optionsSLPK)

[PRT] [2019-09-16 13:01:13] [info] new output directory created at C:/Users/cami9495/Documents/esri-cityengine-sdk-master/examples/py4prt/Demo2_Optimization/../output

[PRT] [2019-09-16 13:01:15] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 13:01:15] [debug] width 256, height 256, rasterCount 3
[PRT] [2019-09-16 13:01:15] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 13:01:15] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 13:01:15] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 13:01:15] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 13:01:15] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 13:01:15] [warning] JPG encoder: ignoring texture channels other than the first 3.
[PRT] [2019-09-16 13:01:16] [debug] width 256, height 512, rasterCount 4
[PRT] [2019-09-16 13:01:16] [debug] width 1024, height 1024, rasterCount 4
[PRT] [2019-09-16 13:01:16] [debug] width 1024, height 1024, rasterCount

In [15]:
item = my_gis.content.add({'type':'Scene Package'}, data=asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
if item is None:
    # first delete the existing one if any and add
    my_gis.content.delete_items([item])
    item = my_gis.content.add({'type':'Scene Package'}, asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
published_item = item.publish()

In [20]:
source = 'http://zurich.maps.arcgis.com/home/webscene/viewer.html?layers=' + published_item.id
IFrame(src=source, width=900, height=600)

-------------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------------------

Step 2 : optimize the total green area reported including the balconies trees. This will be optimized taking into account several parameters:
- the lot coverage
- the height of the first tier

### The variables

In [25]:
goal_str_green = "Greenery Area" + "_sum"

input_attr_green = ["lot_coverage_parameter", "height_first_tier"]
x0_green = [float(80), float(9)]

In [26]:
attrs_green = {'ruleFile' : "rules/typology/envelope2409.cgb", 'startRule' : "Default$Lot", 'report_but_not_display_green' : True}

model2 = m.generate_model(attrs_green, rpk_green, "com.esri.prt.examples.PyEncoder", {'emitGeometry' : False})

[PRT] [2019-09-24 14:12:20] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope2409.rpk

[PRT] [2019-09-24 14:12:20] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/lawn2.jpg" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/lawn2.jpg" type="str"/>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Loader.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Loader.cga" type="str"/>
	<attribute key="/green_optimization_3/assets/vertical_greenery/2.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/vertical_greenery/2.PNG" type="str"/>
	<attribute key="assets/bench.obj

In [27]:
visualize_PRT_results(model2)


Number of generated geometries (= nber of initial shapes):
1

Initial Shape Index: 0

Report of the generated model:
{'Greenery Area_avg': 3.699304017412569, 'Greenery Area_max': 408.2737197150759, 'Greenery Area_min': 0.4895963195850646, 'Greenery Area_n': 574.0, 'Greenery Area_sum': 2123.4005059948145, 'Number of trees_avg': 1.0, 'Number of trees_max': 1.0, 'Number of trees_min': 1.0, 'Number of trees_n': 114.0, 'Number of trees_sum': 114.0, 'green area_avg': 3.3769576217278554, 'green area_max': 408.2737197150759, 'green area_min': 0.4895963195850646, 'green area_n': 460.0, 'green area_sum': 1553.4005059948136}



In [28]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    attrs_green[input_attr_green[1]] = x[1]
    generated_mod_green = m.generate_model(attrs_green)
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green[0])
        return -rep_green[goal_str_green]
    else:
        print("Error in optimization process.")

In [29]:
bounds_green = [(0,100),(8,13)]

In [30]:
-fct_green(x0_green)

2112.7263646109823

In [31]:
start = time.time()

res_green = optimize.differential_evolution(fct_green, bounds_green, seed = 666) #dual_annealing

duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

The optimization lasted: 157.5194342136383


     fun: -2221.57314995954
 message: 'Optimization terminated successfully.'
    nfev: 183
     nit: 5
 success: True
       x: array([99.08078256, 12.79276757])

In [19]:
attrs_green[input_attr_green[0]] = res_green.x[0]
attrs_green[input_attr_green[1]] = res_green.x[1]
green_model2 = m.generate_model(attrs_green)

if green_model2:
    visualize_PRT_results(green_model2)


Number of generated geometries (= nber of initial shapes):
1

Initial Shape Index: 0

Report of the generated model:
{'Number of trees_avg': 1.0, 'Number of trees_max': 1.0, 'Number of trees_min': 1.0, 'Number of trees_n': 138.0, 'Number of trees_sum': 138.0, 'green area_avg': 2.5744739318515535, 'green area_max': 193.4288426155822, 'green area_min': 0.4895976928733391, 'green area_n': 599.0, 'green area_sum': 1542.1098851790805, 'total report for optimisation_avg': 3.964870943255197, 'total report for optimisation_max': 193.4288426155822, 'total report for optimisation_min': 0.4895976928733391, 'total report for optimisation_n': 737.0, 'total report for optimisation_sum': 2922.1098851790803}



### Results Visualization

In [21]:
export_file_name = "SLPKModel2"
enc_optionsSLPK = {'sceneType' : "Local",'baseName' : export_file_name,'sceneWkid' : "3857",'layerTextureEncoding' : ["2"],'layerEnabled' : [True],'layerUID' : ["1"],'layerName' : ["1"],'layerTextureQuality' : [1.0],'layerTextureCompression' : [9],'layerTextureScaling' : [1.0],'layerTextureMaxDimension' : [2048],'layerFeatureGranularity' : ["0"],'layerBackfaceCulling' : [False]}

#attrs_green[input_attr_green[0]] = res_green.x[0]
#attrs_green[input_attr_green[1]] = res_green.x[1]
green_model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.codecs.I3SEncoder", enc_optionsSLPK)

[PRT] [2019-09-16 15:24:22] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:24:22] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:24:22] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:24:22] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:24:22] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:24:22] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:24:22] [debug] width 256, height 256, rasterCount 3
[PRT] [2019-09-16 15:24:22] [warning] JPG encoder: ignoring texture channels other than the first 3.
[PRT] [2019-09-16 15:24:23] [debug] width 256, height 512, rasterCount 4
[PRT] [2019-09-16 15:24:23] [debug] width 1024, height 1024, rasterCount 4
[PRT] [2019-09-16 15:24:23] [debug] width 1024, height 1024, rasterCount 4


In [22]:
item = my_gis.content.add({'type':'Scene Package'}, data=asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
if item is None:
    # first delete the existing one if any and add
    my_gis.content.delete_items([item])
    item = my_gis.content.add({'type':'Scene Package'}, asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
published_item = item.publish()

In [23]:
source2 = 'http://zurich.maps.arcgis.com/home/webscene/viewer.html?layers=' + published_item.id
IFrame(src=source2, width=900, height=600)

-------------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------------------

Step 3 : optimize the total green area reported including the balconies trees. This will be optimized taking into account several parameters:
- the shape of the building (1 = orthogonal, 2 = L shape, 3 = U shape, 4 = O shapes)
- the lot coverage
- the height of the first tier

### The variables

In [36]:
goal_str_green = "Greenery Area" + "_sum"

input_attr_green = ["lot_coverage_parameter", "height_first_tier", "shape_of_building"]
x0_green = [float(50), float(9), float(int(2))]

In [43]:
attrs_green = {'ruleFile' : "rules/typology/envelope2409.cgb", 'startRule' : "Default$Lot", 'report_but_not_display_green' : True}

model3 = m.generate_model(attrs_green, rpk_green, "com.esri.prt.examples.PyEncoder", {'emitGeometry' : False})

[PRT] [2019-09-24 14:54:43] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope2409.rpk

[PRT] [2019-09-24 14:54:43] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/lawn2.jpg" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/lawn2.jpg" type="str"/>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Loader.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Loader.cga" type="str"/>
	<attribute key="/green_optimization_3/assets/vertical_greenery/2.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/vertical_greenery/2.PNG" type="str"/>
	<attribute key="assets/bench.obj

In [44]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    attrs_green[input_attr_green[1]] = x[1]
    attrs_green[input_attr_green[2]] = float(round(x[2]))
    generated_mod_green = m.generate_model(attrs_green)
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green[0])
        return -rep_green[goal_str_green]
    else:
        print("Error in optimization process.")

In [45]:
bounds_green = [(0,100),(8,13),(1,4)]

In [46]:
-fct_green(x0_green)

2566.649574933355

In [47]:
start = time.time()

res_green = optimize.differential_evolution(fct_green, bounds_green, seed = 666) #dual_annealing

duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

[PRT] [2019-09-24 15:17:10] [warning] Rule 'Default$volume$Slab_texture()V' InitialShape '<?>': illegal component geometry, no shape created.
[PRT] [2019-09-24 15:17:10] [warning] Rule 'Default$volume$Slab_texture()V' InitialShape '<?>': illegal component geometry, no shape created.
[PRT] [2019-09-24 15:25:09] [warning] Rule 'Default$volume$Slab_texture()V' InitialShape '<?>': illegal component geometry, no shape created.
[PRT] [2019-09-24 15:25:09] [warning] Rule 'Default$volume$Slab_texture()V' InitialShape '<?>': illegal component geometry, no shape created.
The optimization lasted: 2234.639107465744


     fun: -4230.006530381527
 message: 'Optimization terminated successfully.'
    nfev: 1309
     nit: 28
 success: True
       x: array([33.13302682, 12.80920399,  3.51896445])

In [13]:
attrs_green[input_attr_green[0]] = res_green.x[0]
attrs_green[input_attr_green[1]] = res_green.x[1]
attrs_green[input_attr_green[2]] = float(round(res_green.x[2]))
green_model3 = m.generate_model(attrs_green)

if green_model3:
    visualize_PRT_results(green_model3)


Number of generated geometries (= nber of initial shapes):
1

Initial Shape Index: 0

Report of the generated model:
{'Number of trees_avg': 1.0, 'Number of trees_max': 1.0, 'Number of trees_min': 1.0, 'Number of trees_n': 236.0, 'Number of trees_sum': 236.0, 'green area_avg': 2.6817522843183452, 'green area_max': 265.1220790890036, 'green area_min': 0.33326814578156816, 'green area_n': 833.0, 'green area_sum': 2233.8996528371817, 'total report for optimisation_avg': 4.297380404899138, 'total report for optimisation_max': 265.1220790890036, 'total report for optimisation_min': 0.33326814578156816, 'total report for optimisation_n': 1069.0, 'total report for optimisation_sum': 4593.899652837178}



### Results Visualization

In [14]:
export_file_name = "SLPKModel3"
enc_optionsSLPK = {'sceneType' : "Local",'baseName' : export_file_name,'sceneWkid' : "3857",'layerTextureEncoding' : ["2"],'layerEnabled' : [True],'layerUID' : ["1"],'layerName' : ["1"],'layerTextureQuality' : [1.0],'layerTextureCompression' : [9],'layerTextureScaling' : [1.0],'layerTextureMaxDimension' : [2048],'layerFeatureGranularity' : ["0"],'layerBackfaceCulling' : [False]}

#attrs_green[input_attr_green[0]] = res_green.x[0]
#attrs_green[input_attr_green[1]] = res_green.x[1]
#attrs_green[input_attr_green[2]] = res_green.x[2]
green_model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.codecs.I3SEncoder", enc_optionsSLPK)

[PRT] [2019-09-16 15:49:42] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:49:42] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:49:42] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:49:42] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:49:42] [debug] width 256, height 256, rasterCount 3
[PRT] [2019-09-16 15:49:42] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:49:42] [debug] width 512, height 512, rasterCount 3
[PRT] [2019-09-16 15:49:42] [warning] JPG encoder: ignoring texture channels other than the first 3.
[PRT] [2019-09-16 15:49:43] [debug] width 256, height 512, rasterCount 4
[PRT] [2019-09-16 15:49:43] [debug] width 1024, height 1024, rasterCount 4
[PRT] [2019-09-16 15:49:43] [debug] width 1024, height 1024, rasterCount 4


In [15]:
item = my_gis.content.add({'type':'Scene Package'}, data=asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
if item is None:
    # first delete the existing one if any and add
    my_gis.content.delete_items([item])
    item = my_gis.content.add({'type':'Scene Package'}, asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
published_item = item.publish()

In [16]:
source3 = 'http://zurich.maps.arcgis.com/home/webscene/viewer.html?layers=' + published_item.id
IFrame(src=source3, width=900, height=600)

In [14]:
print("\nShutdown PRT.")
pyprt.shutdown_prt()


Shutdown PRT.
